In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
import datetime

import tensorflow as tf
import tensorflow_hub as hub

!pip install bert-tensorflow --quiet
from bert import tokenization


from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

sys.path.append("./gdrive/My Drive/NLU/Projects/project_2/")

W0611 09:08:25.167742 139641838020480 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


     |████████████████████████████████| 71kB 2.8MB/s 
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
!pip show tensorflow

Name: tensorflow
Version: 1.13.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: tensorboard, grpcio, six, keras-applications, protobuf, numpy, wheel, astor, keras-preprocessing, absl-py, tensorflow-estimator, gast, termcolor
Required-by: stable-baselines, magenta, fancyimpute


In [0]:
flags = tf.flags
FLAGS = flags.FLAGS

flags.DEFINE_string("data_dir", "./gdrive/My Drive/NLU/Projects/project_2/data_pp/", 
                    "The input data dir. Should contain the .tsv files (or other data files).")
flags.DEFINE_string("output_dir", "./output_F/", "The output directory where the model checkpoints will be written.")
flags.DEFINE_string("results_dir", "./results_F/", "The output directory for result files.")

# Pretrained BERT from hub
flags.DEFINE_string("bert_model_hub", "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1", "Pretrained BERT. (default)")
flags.DEFINE_string("bert_dir", None, "Local directory of pretrained BERT.")
flags.DEFINE_bool("bert_trainable", True, "Whether BERT weights are trainable.")
flags.DEFINE_string("init_checkpoint", None, "Initial checkpoint of BERT.")
flags.DEFINE_string("init_checkpoint_sent", None, "Initial checkpoint of pre-trained sentiment model.")

# Pretrained Masked LM BERT
# flags.DEFINE_string("bert_model_hub", None, "Pretrained BERT. (default)")
# flags.DEFINE_string("bert_dir", "./gdrive/My Drive/NLU/Projects/project_2/bert/", "Local directory of pretrained BERT.")
# flags.DEFINE_bool("bert_trainable", True, "Whether BERT weights are trainable.")
# flags.DEFINE_string("init_checkpoint", "./gdrive/My Drive/NLU/Projects/project_2/bert/bert_model.ckpt", "Initial checkpoint of BERT.")
# flags.DEFINE_string("init_checkpoint", "./gdrive/My Drive/NLU/Projects/project_2/bert_masked_lm_pp/", "Initial checkpoint of BERT.")

# Training or evaluation
flags.DEFINE_bool("do_train", False, "Whether to run training.")
flags.DEFINE_bool("do_eval", True, "Whether to run eval on the dev set.")
flags.DEFINE_bool("do_predict", True, "Whether to run the model in inference mode on the test set.")

# Model selection
flags.DEFINE_bool("sentiment_only", False, "sentiment only")
flags.DEFINE_bool("commonsense_only", False, "commonsense only")
flags.DEFINE_bool("combination", False, "combination")

flags.DEFINE_integer("max_seq_length", 400, "Max. length after tokenization. If shorter padded, else truncated.")
flags.DEFINE_integer("batch_size", 8, "Total batch size for training.")
flags.DEFINE_float("learning_rate", 2e-5, "The initial learning rate for Adam.")
flags.DEFINE_float("num_train_epochs", 1.0, "Total number of training epochs to perform.")
flags.DEFINE_float("warmup_proportion", 0.1, "Proportion of training to perform linear learning rate warmup for.")
tf.app.flags.DEFINE_string('f', '', 'tensorflow bug')

if not os.path.exists(FLAGS.output_dir):
    os.makedirs(FLAGS.output_dir)
    
if not os.path.exists(FLAGS.results_dir):
    os.makedirs(FLAGS.results_dir)
    
timestamp = datetime.datetime.now().strftime("%d_%H-%M")

from bert_sct_utils_v2 import create_tokenizer_from_hub_module, SctProcessor, get_config
from bert_sct_utils_v2 import model_fn_builder, file_based_input_fn_builder, file_based_convert_examples_to_features

In [0]:
if not(FLAGS.bert_model_hub or (FLAGS.bert_dir and FLAGS.init_checkpoint)):
    raise ValueError("bert_model_hub or (bert_config_file and init_checkpoint)")

# ---------------------------------------------------------------------------------------------------------------- #
# Create output directory
tf.gfile.MakeDirs(FLAGS.output_dir)

# ---------------------------------------------------------------------------------------------------------------- #
# Processor
processor = SctProcessor()
label_list = processor.get_labels()

bert_config = None
if not FLAGS.init_checkpoint:
    tokenizer = create_tokenizer_from_hub_module(FLAGS.bert_model_hub)

else:
    tokenizer = tokenization.FullTokenizer(vocab_file=os.path.join(FLAGS.bert_dir, "vocab.txt"),
                                           do_lower_case=False)
    bert_config = get_config(os.path.join(FLAGS.bert_dir, "bert_config.json"))

# ---------------------------------------------------------------------------------------------------------------- #
# Model and Estimator
train_examples = None
num_train_steps = None
num_warmup_steps = None
if FLAGS.do_train:
    train_examples = processor.get_train_examples(FLAGS.data_dir)
    num_train_steps = int(len(train_examples) / FLAGS.batch_size * FLAGS.num_train_epochs)
    num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

run_config = tf.estimator.RunConfig(model_dir=FLAGS.output_dir,
                                    save_summary_steps=10,
                                    save_checkpoints_steps=1000)

model_fn = model_fn_builder(bert_model_hub=FLAGS.bert_model_hub,
                            bert_trainable=FLAGS.bert_trainable,
                            bert_config=bert_config,
                            init_checkpoint=FLAGS.init_checkpoint,
                            init_checkpoint_sent=FLAGS.init_checkpoint_sent,
                            num_labels=len(label_list),
                            learning_rate=FLAGS.learning_rate,
                            num_train_steps=num_train_steps,
                            num_warmup_steps=num_warmup_steps,
                            sentiment_only=FLAGS.sentiment_only,
                            commonsense_only=FLAGS.commonsense_only,
                            combination=FLAGS.combination,
                            )


params = {
    "batch_size": FLAGS.batch_size,
}

estimator = tf.estimator.Estimator(model_fn=model_fn,
                                   config=run_config,
                                   params=params)

# ---------------------------------------------------------------------------------------------------------------- #
# Training
if FLAGS.do_train:
    train_file = os.path.join(FLAGS.data_dir, "train.tf_record")
    file_based_convert_examples_to_features(train_examples,
                                            label_list,
                                            FLAGS.max_seq_length,
                                            tokenizer,
                                            train_file)

    tf.logging.info("***** Running training *****")
    tf.logging.info("  Num examples = %d", len(train_examples))
    tf.logging.info("  Batch size = %d", FLAGS.batch_size)
    tf.logging.info("  Num steps = %d", num_train_steps)

    train_input_fn = file_based_input_fn_builder(input_file=train_file,
                                                 seq_length=FLAGS.max_seq_length,
                                                 is_training=True,
                                                 drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

# ---------------------------------------------------------------------------------------------------------------- #
# Evaluation
if FLAGS.do_eval:
    eval_examples = processor.get_dev_examples(FLAGS.data_dir)

    eval_file = os.path.join(FLAGS.data_dir, "eval.tf_record")
    file_based_convert_examples_to_features(eval_examples,
                                            label_list,
                                            FLAGS.max_seq_length,
                                            tokenizer,
                                            eval_file)

    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Num examples = %d", len(eval_examples))

    eval_steps = int(len(eval_examples) // FLAGS.batch_size)

    eval_input_fn = file_based_input_fn_builder(input_file=eval_file,
                                                seq_length=FLAGS.max_seq_length,
                                                is_training=False,
                                                drop_remainder=False)

    result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

    output_eval_file = os.path.join(FLAGS.results_dir, timestamp + "eval_results.txt")
    with tf.gfile.GFile(output_eval_file, "w") as writer:
        tf.logging.info("***** Eval results *****")
        for key in sorted(result.keys()):
            tf.logging.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

# ---------------------------------------------------------------------------------------------------------------- #
# Test
if FLAGS.do_predict:
    predict_examples = processor.get_test_examples(FLAGS.data_dir)

    predict_file = os.path.join(FLAGS.data_dir, "predict.tf_record")
    file_based_convert_examples_to_features(predict_examples,
                                            label_list,
                                            FLAGS.max_seq_length,
                                            tokenizer,
                                            predict_file)

    tf.logging.info("***** Running prediction*****")
    tf.logging.info("  Num examples = %d", len(predict_examples))

    predict_input_fn = file_based_input_fn_builder(input_file=predict_file,
                                                   seq_length=FLAGS.max_seq_length,
                                                   is_training=False,
                                                   drop_remainder=False)

    result = estimator.predict(input_fn=predict_input_fn)

    output_predict_file = os.path.join(FLAGS.results_dir, timestamp + "test_results.tsv")
    with tf.gfile.GFile(output_predict_file, "w") as writer:
        num_written_lines = 0
        tf.logging.info("***** Predict results *****")
        for (i, prediction) in enumerate(result):
            probabilities = prediction["probabilities"]
            predict_label = str(prediction["predict_labels"])

            probs = ["{0:.4f}".format(class_probability) for class_probability in probabilities]

            output_line = "\t".join(probs) + "\t" + predict_label + "\n"
            writer.write(output_line)
            num_written_lines += 1

Instructions for updating:
Colocations handled automatically by placer.


W0610 04:38:30.598176 139670860904320 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0610 04:38:32.737527 139670860904320 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Using config: {'_model_dir': './output_F/', '_tf_random_seed': None, '_save_summary_steps': 10, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f07438d0b00>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


...

I0610 04:41:16.987835 139670860904320 monitored_session.py:222] Graph was finalized.


INFO:tensorflow:Running local_init_op.


I0610 04:41:18.168051 139670860904320 session_manager.py:491] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0610 04:41:18.327720 139670860904320 session_manager.py:493] Done running local_init_op.


In [0]:
import shutil
# shutil.rmtree("./results")
# shutil.rmtree("./output")


shutil.move("./output/", "./output_A/")
# shutil.move("./results/", "./results_A/")

'./output_A/'

In [0]:
FLAGS.do_train = True

In [0]:
FLAGS.data_dir = "./gdrive/My Drive/NLU/Projects/project_2/data_pp_test/"

In [0]:
shutil.move("./results_D/", "./gdrive/My Drive/NLU/Projects/project_2/results/results_D/"